In [1]:
import pandas as pd
# pandas show all columns of table instead of restricted#
pd.set_option('display.max_columns', None)
import geopandas as gpd

import time
import datetime

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig

from eda_quality import correction as correct
from eda_quality import manipulation as manipulate
from eda_quality import inspection as inspect

config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

In [2]:
bbox = BboxSelector([
    7.554130554199218, # min_x
    51.95590322041212, # min_y
    7.590351104736328, # max_x
    51.97874790276371  # max_y
])

track_df = track_api.get_tracks(bbox=bbox, num_results=20) 


# Flag and delete implausible tracks/cases from dataframe

### Track durations

Add a column which holds the time duration of a track. This will be calculated based on the eC variables track.begin and track.end

In [3]:
dfTime=correct.track_duration_time(track_df)
dfTime

,id,time,geometry,Engine Load.value,Engine Load.unit,Calculated MAF.value,Calculated MAF.unit,Speed.value,Speed.unit,CO2.value,CO2.unit,Intake Pressure.value,Intake Pressure.unit,Rpm.value,Rpm.unit,Intake Temperature.value,Intake Temperature.unit,Consumption (GPS-based).value,Consumption (GPS-based).unit,GPS Altitude.value,GPS Altitude.unit,Throttle Position.value,Throttle Position.unit,GPS Bearing.value,GPS Bearing.unit,Consumption.value,Consumption.unit,GPS Accuracy.value,GPS Accuracy.unit,CO2 Emission (GPS-based).value,CO2 Emission (GPS-based).unit,GPS Speed.value,GPS Speed.unit,track.id,track.length,track.begin,track.end,sensor.type,sensor.engineDisplacement,sensor.model,sensor.id,sensor.fuelType,sensor.constructionYear,sensor.manufacturer,track.appVersion,track.touVersion,GPS HDOP.value,GPS HDOP.unit,GPS PDOP.value,GPS PDOP.unit,GPS VDOP.value,GPS VDOP.unit,time_track_begin,time_track_end,track_duration_h
0,5f0ef89c00375c5a2641ef86,2020-07-15T12:37:03+00:00,POINT (7.57939 51.96766),30.459892,%,3.113889,g/s,15.978930,km/h,2.405470,kg/h,29.667201,kPa,748.952252,u/min,26.000000,c,0.936199,l/h,115.671012,m,13.000000,%,136.590329,deg,1.023604,l/h,6.000000,%,2.200068,kg/h,17.502660,km/h,5f0ef89c00375c5a2641ef84,0.665466,2020-07-15T12:37:03Z,2020-07-15T12:38:25Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 12:37:03+00:00,2020-07-15 12:38:25+00:00,00:01:22
1,5f0ef89c00375c5a2641ef88,2020-07-15T12:37:09+00:00,POINT (7.57955 51.96757),49.230105,%,9.778811,g/s,11.134565,km/h,7.554102,kg/h,47.232322,kPa,1475.604745,u/min,25.652838,c,0.937457,l/h,115.280639,m,16.642229,%,134.479803,deg,3.214511,l/h,6.000000,%,2.203025,kg/h,10.324164,km/h,5f0ef89c00375c5a2641ef84,0.665466,2020-07-15T12:37:03Z,2020-07-15T12:38:25Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 12:37:03+00:00,2020-07-15 12:38:25+00:00,00:01:22
2,5f0ef89c00375c5a2641ef89,2020-07-15T12:37:14+00:00,POINT (7.57988 51.96740),78.649652,%,25.066406,g/s,33.976330,km/h,19.363722,kg/h,74.040426,kPa,2397.395931,u/min,23.728013,c,5.102906,l/h,114.613231,m,23.862069,%,123.313954,deg,8.239881,l/h,6.193485,%,11.991830,kg/h,30.967132,km/h,5f0ef89c00375c5a2641ef84,0.665466,2020-07-15T12:37:03Z,2020-07-15T12:38:25Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 12:37:03+00:00,2020-07-15 12:38:25+00:00,00:01:22
3,5f0ef89c00375c5a2641ef8a,2020-07-15T12:37:19+00:00,POINT (7.58049 51.96715),31.200400,%,7.419664,g/s,39.000000,km/h,5.731668,kg/h,30.075758,kPa,1748.565672,u/min,24.000000,c,2.280491,l/h,113.835218,m,15.351261,%,125.035780,deg,2.439007,l/h,6.000000,%,5.359154,kg/h,36.898346,km/h,5f0ef89c00375c5a2641ef84,0.665466,2020-07-15T12:37:03Z,2020-07-15T12:38:25Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 12:37:03+00:00,2020-07-15 12:38:25+00:00,00:01:22
4,5f0ef89c00375c5a2641ef8b,2020-07-15T12:37:24+00:00,POINT (7.58107 51.96682),29.735773,%,3.106679,g/s,34.321667,km/h,2.399900,kg/h,29.000000,kPa,761.854074,u/min,25.000000,c,0.946257,l/h,113.502384,m,13.000000,%,133.482068,deg,1.021234,l/h,7.489919,%,2.223704,kg/h,34.447545,km/h,5f0ef89c00375c5a2641ef84,0.665466,2020-07-15T12:37:03Z,2020-07-15T12:38:25Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 12:37:03+00:00,2020-07-15 12:38:25+00:00,00:01:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,5de9d7a63bdb691868e77f3d,2019-11-28T07:04:47+00:00,POINT (7.65198 51.93501),54.451786,%,11.638098,g/s,2.592391,km/h,8.990395,kg/h,54.574124,kPa,1455.551457,u/min,13.000000,c,NaN,NaN,101.874041,m,16.188888,%,33.423610,deg,3.825700,l/h,4.000000,%,NaN,NaN

### Tracks exceeding duration of 8 hours
The following method will give you three dataframes: <br>
<li> Complete geopandas input dataframe with added column which flags all cases belonging to a track longer than 8 hours
<li> Geopandas dataframe free off tracks longer than 8 hours
<li> Pandas dataframe with only two columns: track.id and track duration

In [4]:
dfOr, dfEightclean, dfEight  = correct.exceed_eight_hours(track_df)

no track duration exceeds eight hours


### Tracks falling below 5 min time duration
The following method will give you three dataframes: <br>
<li> Complete geopandas input dataframe with added column which flags all cases belonging to a track shorter than 5 min
<li> Geopandas dataframe free off tracks shorter than 5 min
<li> Pandas dataframe with only two columns: track.id and track duration

In [5]:
dfOrig, dfFiveclean, dfFive  = correct.below_five_min(track_df)

13 tracks are shorter than 5 minutes




# Flag and delete implausible speeds

### Observations with speed > 250km/h
The following method will give you three dataframes: <br>
<li> Complete geopandas input dataframe with added column which flags all cases with speed > 250km/h
<li> Geopandas dataframe free off tracks with max speed > 250 km/h
<li> Pandas dataframe with only two columns: track.id and track_max_speed

In [9]:
dfOri, cleanDF, df_250 = correct.implausible_Max_Speed(track_df)

no track exceeds max speed 250km/h
